In [11]:
import pandas as pd
import requests
import tempfile
import certifi

# URL for Penn World Table data (modify according to the specific version you're using)
url = 'https://www.rug.nl/ggdc/docs/pwt90.dta'
response = requests.get(url, verify=certifi.where())
response.raise_for_status()

# Save to a temporary file
with tempfile.NamedTemporaryFile(delete=False, suffix='.dta') as tmp:
    tmp.write(response.content)
    tmp_path = tmp.name

# Load the data using pandas
pwt_data = pd.read_stata(tmp_path)

# Filter data for the period 1960-2000 and specific countries if needed
pwt_filtered = pwt_data[(pwt_data['year'] >= 1960) & (pwt_data['year'] <= 2000)]


In [19]:
# Filter the relevant years and countries
countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 
             'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 
             'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 
             'United Kingdom', 'United States']  # Add or remove based on availability

pwt_filtered = pwt_data[(pwt_data['year'] >= 1960) & (pwt_data['year'] <= 2000) & 
                        (pwt_data['country'].isin(countries))]

# Calculate average annual growth rates
growth_rates = pwt_filtered.groupby('country').apply(lambda x: 
                                                     (x[['rgdpna', 'rtfpna', 'rkna']].pct_change() + 1).prod() ** (1/len(x)) - 1)


/var/folders/_1/6pq414md0y1_mgwvl5dlnd2h0000gn/T/ipykernel_78117/147861487.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  growth_rates = pwt_filtered.groupby('country').apply(lambda x:


In [20]:
# Initialize a DataFrame for the results
results = pd.DataFrame(index=countries, columns=['Growth Rate', 'TFP Growth', 'Capital Deepening', 'TFP Share', 'Capital Share'])

# Fill in the calculated data
results['Growth Rate'] = growth_rates['rgdpna'] * 100  # Convert to percentage
results['TFP Growth'] = growth_rates['rtfpna'] * 100
results['Capital Deepening'] = growth_rates['rkna'] * 100

# Calculate contributions as shares
results['TFP Share'] = results['TFP Growth'] / results['Growth Rate']
results['Capital Share'] = results['Capital Deepening'] / results['Growth Rate']

# Display the results
print(results)


                Growth Rate  TFP Growth  Capital Deepening  TFP Share  \
Australia          3.592646    0.838780           3.561521   0.233471   
Austria            3.205478    1.408362           4.043484   0.439361   
Belgium            3.057003    1.636624           3.129745   0.535369   
Canada             3.557873    0.617492           4.061985   0.173557   
Denmark            2.895236    1.098835           3.975713   0.379532   
Finland            3.413653    1.711261           3.926492   0.501299   
France             3.393149    1.852989           4.176021   0.546097   
Germany            2.826917    1.656210           3.740466   0.585872   
Greece             3.918755    0.970745           4.950404   0.247718   
Iceland            3.966475    1.001704           4.331708   0.252543   
Ireland            4.614973    1.892865           4.104078   0.410157   
Italy              3.461969    1.404321           4.101086   0.405642   
Japan              4.916239    0.845432          11